# TensorFlow__MLP_MNIST-labeled100__pollenjp

- 論文
  - [Semi-Supervised Learning with Ladder Networks - Antti Rasmus, Harri Valpola, Mikko Honkala, Mathias Berglund, Tapani Raiko](https://arxiv.org/abs/1507.02672)

- code
  - [rinuboney/ladder](https://github.com/rinuboney/ladder)
  - [tarvaina/tensorflow-ladder](https://github.com/tarvaina/tensorflow-ladder)


<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Config" data-toc-modified-id="Config-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Config</a></span><ul class="toc-item"><li><span><a href="#Import" data-toc-modified-id="Import-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Import</a></span></li><li><span><a href="#TensorFlow,-Keras" data-toc-modified-id="TensorFlow,-Keras-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>TensorFlow, Keras</a></span></li><li><span><a href="#Path" data-toc-modified-id="Path-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Path</a></span></li><li><span><a href="#Import-from-Local" data-toc-modified-id="Import-from-Local-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Import from Local</a></span></li></ul></li><li><span><a href="#Load-MNIST" data-toc-modified-id="Load-MNIST-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load MNIST</a></span></li><li><span><a href="#Simple-MLP" data-toc-modified-id="Simple-MLP-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Simple MLP</a></span><ul class="toc-item"><li><span><a href="#fit" data-toc-modified-id="fit-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>fit</a></span></li></ul></li><li><span><a href="#MLP-(Batch-Normalization)" data-toc-modified-id="MLP-(Batch-Normalization)-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>MLP (Batch Normalization)</a></span></li><li><span><a href="#show-Result" data-toc-modified-id="show-Result-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>show Result</a></span></li><li><span><a href="#End" data-toc-modified-id="End-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>End</a></span></li></ul></div>

## Config

### Import

In [1]:
import os,sys
print(sys.version)

import re
from pathlib import Path
import math

3.5.2 (default, Nov 23 2017, 16:37:01) 
[GCC 5.4.0 20160609]


In [2]:
# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload
# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2
# If you want to reload manually, add a below line head.
%aimport
# ref: https://ipython.org/ipython-doc/3/config/extensions/autoreload.html

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import sklearn
from sklearn import datasets
import tqdm
import csv
import pandas as pd

seed = None
np.random.seed(seed=seed)

print("numpy ver: {}".format(np.__version__))
print("scikit-learn ver: {}".format(sklearn.__version__))
print("pandas ver: {}".format(pd.__version__))

Modules to reload:
all-except-skipped

Modules to skip:

numpy ver: 1.15.1
scikit-learn ver: 0.19.2
pandas ver: 0.23.4


### TensorFlow, Keras

In [3]:
#____________________________________________________________________________________________________
#  TensorFlow and Keras GPU configures
##________________________________________________________________________________
##  OPTIONAL : set a GPU viewed by TensorFlow
###____________________________________________________________
###  - https://stackoverflow.com/questions/37893755/tensorflow-set-cuda-visible-devices-within-jupyter
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"
##________________________________________________________________________________


##________________________________________________________________________________
##  TensorFlow
###____________________________________________________________
import tensorflow as tf
print("tensorflow ver: {}".format(tf.__version__))
###  eager mode
#tf.enable_eager_execution()
print("tf.executing_eagerly(): {}".format(tf.executing_eagerly()))

# You can double check that you have the correct devices visible to TF
#   - https://stackoverflow.com/questions/37893755/tensorflow-set-cuda-visible-devices-within-jupyter
from tensorflow.python.client import device_lib
print("""
________________________________________
Visible GPUs from TensorFlow
________________________________________""")
for _device in device_lib.list_local_devices():
    match = re.search(pattern=r'name: "/device:(?P<name>[A-Z]{3}):(?P<device_num>\d{1})*',
                      string=str(_device))
    if match is None:
        print("Not Match")
        continue
    if match.group("name") == "CPU":
        name, device_num = match.group("name", "device_num")
        print()
        print("({}:{})".format(name, device_num))
        continue
    name, device_num = match.group("name", "device_num")
    match = re.search(pattern=r'.*pci bus id: (?P<pci_bus_id>\d{4}:\d{2}:\d{2}.\d{1}).*',
                      string=str(_device))
    if match is None:
        print("No GPUs")
        continue
    print("({}:{}: pci_bus_id: {})".format(name, device_num, match.group("pci_bus_id")))
print("________________________________________")

###____________________________________________________________
###  sessioin
global _SESSION
config = tf.ConfigProto(allow_soft_placement=True,
                        log_device_placement=True)
config.gpu_options.allow_growth = True
_SESSION = tf.Session(config=config)
###____________________________________________________________
##________________________________________________________________________________
#____________________________________________________________________________________________________

import keras
import keras.backend as K
config = tf.ConfigProto(allow_soft_placement=True,
                        log_device_placement=True)
config.gpu_options.allow_growth = True
_SESSION = tf.Session(config=config)
K.set_session(_SESSION)

tensorflow ver: 1.10.1
tf.executing_eagerly(): False

________________________________________
Visible GPUs from TensorFlow
________________________________________

(CPU:0)
(GPU:0: pci_bus_id: 0000:02:00.0)
________________________________________


Using TensorFlow backend.


### Path

In [4]:
HOME = Path(os.getcwd()).parent
print(HOME)

/home/pollenjp/workdir/git/article_script/20180914__semi-supervised-deeplearning-ladder-networks__in_kabuku


In [5]:
path_list = []
data_Path = HOME / "data"
path_list.append(data_Path)
raw_Path = data_Path / "raw"
path_list.append(raw_Path)
plot_images_Path = data_Path / "plot_images"
path_list.append(plot_images_Path)
src_Path = HOME / "src"
path_list.append(src_Path)

for _Path in path_list:
    _path = str(_Path)
    if not os.path.exists(_path):
        os.makedirs(name=_path)
        print("make a directory: \n\t", _path)
    else:
        print(os.path.exists(_path), ": ", _path)

True :  /home/pollenjp/workdir/git/article_script/20180914__semi-supervised-deeplearning-ladder-networks__in_kabuku/data
True :  /home/pollenjp/workdir/git/article_script/20180914__semi-supervised-deeplearning-ladder-networks__in_kabuku/data/raw
True :  /home/pollenjp/workdir/git/article_script/20180914__semi-supervised-deeplearning-ladder-networks__in_kabuku/data/plot_images
True :  /home/pollenjp/workdir/git/article_script/20180914__semi-supervised-deeplearning-ladder-networks__in_kabuku/src


### Import from Local

In [6]:
sys.path.append(str(src_Path))

In [7]:
from data import mnist_input_data
from utils_tensorflow.tensorflow_graph_in_jupyer import show_computational_graph

## Load MNIST

In [8]:
num_labeled = 100
print( "===  Loading Data ===" )
mnist = mnist_input_data.read_data_sets(train_dir=str(raw_Path / "MNIST_data"), n_labeled=num_labeled,
                                        fake_data=False, one_hot=True)

===  Loading Data ===
Extracting /home/pollenjp/workdir/git/article_script/20180914__semi-supervised-deeplearning-ladder-networks__in_kabuku/data/raw/MNIST_data/train-images-idx3-ubyte.gz
Extracting /home/pollenjp/workdir/git/article_script/20180914__semi-supervised-deeplearning-ladder-networks__in_kabuku/data/raw/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /home/pollenjp/workdir/git/article_script/20180914__semi-supervised-deeplearning-ladder-networks__in_kabuku/data/raw/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /home/pollenjp/workdir/git/article_script/20180914__semi-supervised-deeplearning-ladder-networks__in_kabuku/data/raw/MNIST_data/t10k-labels-idx1-ubyte.gz


In [9]:
X_train = mnist.train.images
y_train = mnist.train.labels
X_test = mnist.test.images
y_test = mnist.test.labels
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(60000, 784)
(60000, 10)
(10000, 784)
(10000, 10)


In [10]:
y_train[:10]
y_train.sum(axis=0)

array([5923., 6742., 5958., 6131., 5842., 5421., 5918., 6265., 5851.,
       5949.])

## Simple MLP

In [11]:
layer_sizes = [784, 1000, 500, 250, 250, 250, 10]
L = len(layer_sizes) - 1  # number of layers

num_examples = 60000
num_epochs = 50

starter_learning_rate = 0.02
decay_after = 15  # epoch after which to begin learning rate decay
batch_size = 100
num_iter = (num_examples//batch_size) * num_epochs # number of loop iterations

In [12]:
from keras.models import Sequential
from keras import layers
from keras import activations

In [13]:
K.clear_session()
layer_sizes = [784, 1000, 500, 250, 250, 250, 10]
model = Sequential()
model.add(layer=layers.Dense(units=layer_sizes[1], activation="relu", input_shape=(layer_sizes[0], )))
model.add(layer=layers.BatchNormalization())
model.add(layer=layers.Dense(units=layer_sizes[2], activation="relu"))
model.add(layer=layers.BatchNormalization())
model.add(layer=layers.Dense(units=layer_sizes[3], activation="relu"))
model.add(layer=layers.BatchNormalization())
model.add(layer=layers.Dense(units=layer_sizes[4], activation="relu"))
model.add(layer=layers.BatchNormalization())
model.add(layer=layers.Dense(units=layer_sizes[5], activation="relu"))
model.add(layer=layers.BatchNormalization())
model.add(layer=layers.Dense(units=layer_sizes[6], activation="softmax"))

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1000)              785000    
_________________________________________________________________
batch_normalization_1 (Batch (None, 1000)              4000      
_________________________________________________________________
dense_2 (Dense)              (None, 500)               500500    
_________________________________________________________________
batch_normalization_2 (Batch (None, 500)               2000      
_________________________________________________________________
dense_3 (Dense)              (None, 250)               125250    
_________________________________________________________________
batch_normalization_3 (Batch (None, 250)               1000      
_________________________________________________________________
dense_4 (Dense)              (None, 250)               62750     
__________

In [15]:
from keras import optimizers

In [16]:
learning_rate = 0.02
metrics = ["accuracy"]
opt = optimizers.Adam(lr=learning_rate)
model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=metrics)

### fit

In [17]:
model.fit(x=X_train, y=y_train, batch_size=batch_size, epochs=num_epochs,
          validation_data=(X_test[:], y_test[:]) )

Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 5s 77us/step - loss: 0.2668 - acc: 0.9226 - val_loss: 0.1807 - val_acc: 0.9512
Epoch 2/50
60000/60000 [==============================] - 4s 59us/step - loss: 0.1414 - acc: 0.9594 - val_loss: 0.1287 - val_acc: 0.9625
Epoch 3/50
60000/60000 [==============================] - 4s 58us/step - loss: 0.1146 - acc: 0.9670 - val_loss: 0.1109 - val_acc: 0.9687
Epoch 4/50
60000/60000 [==============================] - 4s 59us/step - loss: 0.0945 - acc: 0.9733 - val_loss: 0.1239 - val_acc: 0.9648
Epoch 5/50
60000/60000 [==============================] - 4s 59us/step - loss: 0.0835 - acc: 0.9756 - val_loss: 0.1022 - val_acc: 0.9718
Epoch 6/50
60000/60000 [==============================] - 4s 59us/step - loss: 0.0827 - acc: 0.9762 - val_loss: 0.1311 - val_acc: 0.9652
Epoch 7/50
60000/60000 [==============================] - 4s 59us/step - loss: 0.0672 - acc: 0.9805 - val_loss: 0.0959 - val_acc

In [18]:
show_computational_graph(graph_def=tf.get_default_graph())

## show Result

## End